<a href="https://colab.research.google.com/github/prksh830/Healthcare/blob/main/RUL(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load dataset
data = pd.read_csv('Battery_RUL.csv')  # Replace with your dataset path

# Remove cycle index
if 'cycle' in data.columns:
    data = data.drop(columns=['cycle'])

# Separate features and target
X = data.drop(columns=['RUL'])  # Replace 'RUL' with your target column name
y = data['RUL']

print(f"Initial dataset shape: {X.shape[0]} samples, {X.shape[1]} features")

Initial dataset shape: 15064 samples, 8 features


In [3]:
import pandas as pd

# Load dataset
data = pd.read_csv('Battery_RUL1.csv')  # Replace with your dataset path

# Remove cycle index
if 'cycle' in data.columns:
    data = data.drop(columns=['cycle'])

# Separate features and target
X = data.drop(columns=['RUL'])  # Replace 'RUL' with your target column name
y = data['RUL']

print(f"Initial dataset shape: {X.shape[0]} samples, {X.shape[1]} features")

Initial dataset shape: 15064 samples, 7 features


In [4]:
import numpy as np

# Compute IQR
Q1 = X.quantile(0.25)
Q3 = X.quantile(0.75)
IQR = Q3 - Q1

# Filter out outliers
mask = ~((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).any(axis=1)
X, y = X[mask], y[mask]

print(f"Dataset after outlier removal: {X.shape[0]} samples, {X.shape[1]} features")

Dataset after outlier removal: 14445 samples, 7 features


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")

Training set: 11556 samples
Testing set: 2889 samples


In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
pip install xgboost

In [9]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 12.0 MB/s eta 0:00:00


In [11]:
import xgboost as xgb
import optuna
from sklearn.metrics import mean_squared_error
import numpy as np

def optimize_xgboost(X_train, y_train, n_trials=30):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 5)
        }
        model = xgb.XGBRegressor(**params, random_state=42)
        model.fit(X_train, y_train)
        preds = model.predict(X_train)
        rmse = np.sqrt(mean_squared_error(y_train, preds))
        return rmse

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)
    return study.best_params

xgb_params = optimize_xgboost(X_train.values, y_train)
print("Best XGBoost params:", xgb_params)

[I 2025-09-30 14:05:54,256] A new study created in memory with name: no-name-57f2d123-a64c-45e4-9ecc-eebb5ea71324
[I 2025-09-30 14:05:54,811] Trial 0 finished with value: 11.724029409745864 and parameters: {'n_estimators': 388, 'max_depth': 5, 'learning_rate': 0.15678344095276617, 'subsample': 0.7955732970442503, 'colsample_bytree': 0.6162678962389877, 'gamma': 4.1659522305037475}. Best is trial 0 with value: 11.724029409745864.
[I 2025-09-30 14:05:55,211] Trial 1 finished with value: 14.443554157958047 and parameters: {'n_estimators': 139, 'max_depth': 7, 'learning_rate': 0.0795469272579617, 'subsample': 0.7949973250364671, 'colsample_bytree': 0.7911625538948883, 'gamma': 4.052349107605625}. Best is trial 0 with value: 11.724029409745864.
[I 2025-09-30 14:05:55,750] Trial 2 finished with value: 8.35812773624399 and parameters: {'n_estimators': 362, 'max_depth': 5, 'learning_rate': 0.24834235059301685, 'subsample': 0.9717017587049914, 'colsample_bytree': 0.7216173094888155, 'gamma': 0.

Best XGBoost params: {'n_estimators': 375, 'max_depth': 12, 'learning_rate': 0.22923378492695154, 'subsample': 0.8334273539516317, 'colsample_bytree': 0.9104617149434099, 'gamma': 0.0034110438186890956}


In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import optuna

def optimize_rf(X_train, y_train, n_trials=30):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5)
        }
        model = RandomForestRegressor(**params, random_state=42)
        model.fit(X_train, y_train)
        preds = model.predict(X_train)
        rmse = np.sqrt(mean_squared_error(y_train, preds))
        return rmse

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)
    return study.best_params

rf_params = optimize_rf(X_train.values, y_train)
print("Best Random Forest params:", rf_params)

[I 2025-09-30 14:08:07,331] A new study created in memory with name: no-name-339dbfbf-6ba8-4e71-9afc-aee6b5f7dbd8
[I 2025-09-30 14:08:32,237] Trial 0 finished with value: 6.963128020166952 and parameters: {'n_estimators': 276, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 1}. Best is trial 0 with value: 6.963128020166952.
[I 2025-09-30 14:08:45,043] Trial 1 finished with value: 40.26570955555705 and parameters: {'n_estimators': 385, 'max_depth': 5, 'min_samples_split': 5, 'min_samples_leaf': 3}. Best is trial 0 with value: 6.963128020166952.
[I 2025-09-30 14:08:56,593] Trial 2 finished with value: 25.664607580025073 and parameters: {'n_estimators': 226, 'max_depth': 8, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 0 with value: 6.963128020166952.
[I 2025-09-30 14:09:04,303] Trial 3 finished with value: 16.313044196933475 and parameters: {'n_estimators': 117, 'max_depth': 11, 'min_samples_split': 2, 'min_samples_leaf': 5}. Best is trial 0 with value: 6.963

Best Random Forest params: {'n_estimators': 433, 'max_depth': 18, 'min_samples_split': 2, 'min_samples_leaf': 1}


In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import optuna

def optimize_rf(X_train, y_train, n_trials=30):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5)
        }
        model = RandomForestRegressor(**params, random_state=42)
        model.fit(X_train, y_train)
        preds = model.predict(X_train)
        rmse = np.sqrt(mean_squared_error(y_train, preds))
        return rmse

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)
    return study.best_params

rf_params = optimize_rf(X_train.values, y_train)
print("Best Random Forest params:", rf_params)

[I 2025-09-30 14:47:15,574] A new study created in memory with name: no-name-4fb90d7a-de72-44b5-a1bd-38d138182c93
[I 2025-09-30 14:47:36,003] Trial 0 finished with value: 16.14026240646946 and parameters: {'n_estimators': 317, 'max_depth': 11, 'min_samples_split': 6, 'min_samples_leaf': 5}. Best is trial 0 with value: 16.14026240646946.
[I 2025-09-30 14:47:40,734] Trial 1 finished with value: 45.022376474227435 and parameters: {'n_estimators': 184, 'max_depth': 4, 'min_samples_split': 9, 'min_samples_leaf': 4}. Best is trial 0 with value: 16.14026240646946.
[I 2025-09-30 14:48:00,835] Trial 2 finished with value: 8.35325962886464 and parameters: {'n_estimators': 242, 'max_depth': 17, 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 2 with value: 8.35325962886464.
[I 2025-09-30 14:48:03,629] Trial 3 finished with value: 54.71204590385465 and parameters: {'n_estimators': 139, 'max_depth': 3, 'min_samples_split': 4, 'min_samples_leaf': 5}. Best is trial 2 with value: 8.353259

Best Random Forest params: {'n_estimators': 500, 'max_depth': 20, 'min_samples_split': 3, 'min_samples_leaf': 1}


In [17]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import optuna

def optimize_gb(X_train, y_train, n_trials=30):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0)
        }
        model = GradientBoostingRegressor(**params, random_state=42)
        model.fit(X_train, y_train)
        preds = model.predict(X_train)
        rmse = np.sqrt(mean_squared_error(y_train, preds))
        return rmse

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)
    return study.best_params

gb_params = optimize_gb(X_train.values, y_train)
print("Best Gradient Boosting params:", gb_params)

[I 2025-09-30 15:02:22,346] A new study created in memory with name: no-name-1c03f06c-ee43-4140-9604-56ad4e18a09f
[I 2025-09-30 15:02:31,004] Trial 0 finished with value: 12.824787990161388 and parameters: {'n_estimators': 292, 'max_depth': 4, 'learning_rate': 0.2969199062119607, 'subsample': 0.6437593456347144}. Best is trial 0 with value: 12.824787990161388.
[I 2025-09-30 15:02:37,024] Trial 1 finished with value: 20.577079942724225 and parameters: {'n_estimators': 276, 'max_depth': 3, 'learning_rate': 0.2866014890519411, 'subsample': 0.6769916732272879}. Best is trial 0 with value: 12.824787990161388.
[I 2025-09-30 15:02:48,788] Trial 2 finished with value: 24.245912200168526 and parameters: {'n_estimators': 309, 'max_depth': 4, 'learning_rate': 0.06121340644725708, 'subsample': 0.9057017825637231}. Best is trial 0 with value: 12.824787990161388.
[I 2025-09-30 15:02:53,310] Trial 3 finished with value: 7.449377343163437 and parameters: {'n_estimators': 75, 'max_depth': 8, 'learning_

Best Gradient Boosting params: {'n_estimators': 408, 'max_depth': 12, 'learning_rate': 0.1461312424381085, 'subsample': 0.8736377014399374}


In [21]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import optuna

def optimize_catboost(X_train, y_train, n_trials=30):
    def objective(trial):
        params = {
            'iterations': trial.suggest_int('iterations', 100, 500),
            'depth': trial.suggest_int('depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10)
        }
        model = CatBoostRegressor(**params, random_state=42, verbose=0)
        model.fit(X_train, y_train)
        preds = model.predict(X_train)
        rmse = np.sqrt(mean_squared_error(y_train, preds))
        return rmse

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)
    return study.best_params

cat_params = optimize_catboost(X_train.values, y_train)
print("Best CatBoost params:", cat_params)

[I 2025-09-30 15:17:08,213] A new study created in memory with name: no-name-2c2fb2d5-a604-417d-9eb2-eabcee7b7453
[I 2025-09-30 15:17:21,588] Trial 0 finished with value: 8.616042236949333 and parameters: {'iterations': 462, 'depth': 10, 'learning_rate': 0.29550694026417756, 'l2_leaf_reg': 8.482531678586367}. Best is trial 0 with value: 8.616042236949333.
[I 2025-09-30 15:17:22,055] Trial 1 finished with value: 29.523105340839912 and parameters: {'iterations': 252, 'depth': 3, 'learning_rate': 0.24295227899660063, 'l2_leaf_reg': 4.719686404379984}. Best is trial 0 with value: 8.616042236949333.
[I 2025-09-30 15:17:22,490] Trial 2 finished with value: 26.452103134191223 and parameters: {'iterations': 142, 'depth': 5, 'learning_rate': 0.24429068107785779, 'l2_leaf_reg': 5.4011312188303044}. Best is trial 0 with value: 8.616042236949333.
[I 2025-09-30 15:17:30,423] Trial 3 finished with value: 18.667653234882383 and parameters: {'iterations': 257, 'depth': 10, 'learning_rate': 0.089642565

Best CatBoost params: {'iterations': 453, 'depth': 9, 'learning_rate': 0.2990058709227652, 'l2_leaf_reg': 3.1853562032273657}


In [23]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import numpy as np
import optuna

def optimize_ridge(X_train_scaled, y_train, n_trials=30):
    def objective(trial):
        alpha = trial.suggest_float('alpha', 0.01, 10.0)
        model = Ridge(alpha=alpha, random_state=42)
        model.fit(X_train_scaled, y_train)
        preds = model.predict(X_train_scaled)
        rmse = np.sqrt(mean_squared_error(y_train, preds))
        return rmse

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)
    return study.best_params

ridge_params = optimize_ridge(X_train_scaled, y_train)
print("Best Ridge params:", ridge_params)

[I 2025-09-30 15:20:22,193] A new study created in memory with name: no-name-a5f3578f-d5e1-478e-be4d-b9bb448107b8
[I 2025-09-30 15:20:22,202] Trial 0 finished with value: 46.721464810919095 and parameters: {'alpha': 5.753762635098745}. Best is trial 0 with value: 46.721464810919095.
[I 2025-09-30 15:20:22,208] Trial 1 finished with value: 46.71788809088803 and parameters: {'alpha': 4.021942852893118}. Best is trial 1 with value: 46.71788809088803.
[I 2025-09-30 15:20:22,213] Trial 2 finished with value: 46.717884161762456 and parameters: {'alpha': 4.0197471980394806}. Best is trial 2 with value: 46.717884161762456.
[I 2025-09-30 15:20:22,218] Trial 3 finished with value: 46.73097226120818 and parameters: {'alpha': 9.219225618607977}. Best is trial 2 with value: 46.717884161762456.
[I 2025-09-30 15:20:22,223] Trial 4 finished with value: 46.716703074361405 and parameters: {'alpha': 3.310207041905016}. Best is trial 4 with value: 46.716703074361405.
[I 2025-09-30 15:20:22,228] Trial 5 fi

Best Ridge params: {'alpha': 0.033435008620146256}


In [24]:
from sklearn.linear_model import LinearRegression

# Linear Regression has no hyperparameters to optimize
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train)
print("Linear Regression model trained successfully")

Linear Regression model trained successfully


In [26]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import optuna

def optimize_knn(X_train_scaled, y_train, n_trials=30):
    def objective(trial):
        params = {
            'n_neighbors': trial.suggest_int('n_neighbors', 2, 20),
            'weights': trial.suggest_categorical('weights', ['uniform', 'distance']),
            'p': trial.suggest_int('p', 1, 2)
        }
        model = KNeighborsRegressor(**params)
        model.fit(X_train_scaled, y_train)
        preds = model.predict(X_train_scaled)
        rmse = np.sqrt(mean_squared_error(y_train, preds))
        return rmse

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)
    return study.best_params

knn_params = optimize_knn(X_train_scaled, y_train)
print("Best KNN params:", knn_params)

[I 2025-09-30 15:21:29,491] A new study created in memory with name: no-name-fbfcbbf5-4ff5-469e-85ce-620bd7ab5af4
[I 2025-09-30 15:21:29,649] Trial 0 finished with value: 16.047426975856002 and parameters: {'n_neighbors': 8, 'weights': 'uniform', 'p': 1}. Best is trial 0 with value: 16.047426975856002.
[I 2025-09-30 15:21:29,746] Trial 1 finished with value: 0.09042991813254031 and parameters: {'n_neighbors': 5, 'weights': 'distance', 'p': 2}. Best is trial 1 with value: 0.09042991813254031.
[I 2025-09-30 15:21:29,978] Trial 2 finished with value: 0.09042991813254031 and parameters: {'n_neighbors': 17, 'weights': 'distance', 'p': 1}. Best is trial 1 with value: 0.09042991813254031.
[I 2025-09-30 15:21:30,092] Trial 3 finished with value: 0.09042991813254031 and parameters: {'n_neighbors': 7, 'weights': 'distance', 'p': 2}. Best is trial 1 with value: 0.09042991813254031.
[I 2025-09-30 15:21:30,241] Trial 4 finished with value: 23.903764633084 and parameters: {'n_neighbors': 18, 'weight

Best KNN params: {'n_neighbors': 5, 'weights': 'distance', 'p': 2}


In [27]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

In [29]:
# Train XGBoost with best parameters
xgb_model = xgb.XGBRegressor(**xgb_params, random_state=42)
xgb_model.fit(X_train.values, y_train)
xgb_preds = xgb_model.predict(X_test.values)

# Evaluate
xgb_mae = mean_absolute_error(y_test, xgb_preds)
xgb_mse = mean_squared_error(y_test, xgb_preds)
xgb_rmse = np.sqrt(xgb_mse)
xgb_r2 = r2_score(y_test, xgb_preds)

print("XGBoost Metrics:")
print(f"MAE: {xgb_mae:.4f}, MSE: {xgb_mse:.4f}, RMSE: {xgb_rmse:.4f}, R2: {xgb_r2:.4f}\n")

XGBoost Metrics:
MAE: 8.7412, MSE: 283.7331, RMSE: 16.8444, R2: 0.9972



In [31]:
rf_model = RandomForestRegressor(**rf_params, random_state=42)
rf_model.fit(X_train.values, y_train)
rf_preds = rf_model.predict(X_test.values)

rf_mae = mean_absolute_error(y_test, rf_preds)
rf_mse = mean_squared_error(y_test, rf_preds)
rf_rmse = np.sqrt(rf_mse)
rf_r2 = r2_score(y_test, rf_preds)

print("Random Forest Metrics:")
print(f"MAE: {rf_mae:.4f}, MSE: {rf_mse:.4f}, RMSE: {rf_rmse:.4f}, R2: {rf_r2:.4f}\n")

Random Forest Metrics:
MAE: 8.3264, MSE: 252.5705, RMSE: 15.8925, R2: 0.9975



In [33]:
gb_model = GradientBoostingRegressor(**gb_params, random_state=42)
gb_model.fit(X_train.values, y_train)
gb_preds = gb_model.predict(X_test.values)

gb_mae = mean_absolute_error(y_test, gb_preds)
gb_mse = mean_squared_error(y_test, gb_preds)
gb_rmse = np.sqrt(gb_mse)
gb_r2 = r2_score(y_test, gb_preds)

print("Gradient Boosting Metrics:")
print(f"MAE: {gb_mae:.4f}, MSE: {gb_mse:.4f}, RMSE: {gb_rmse:.4f}, R2: {gb_r2:.4f}\n")

Gradient Boosting Metrics:
MAE: 7.7027, MSE: 264.2629, RMSE: 16.2562, R2: 0.9974



In [34]:
cat_params['iterations'] = 50  # Set iterations to 50
cat_model = CatBoostRegressor(**cat_params, random_state=42, verbose=0)
cat_model.fit(X_train.values, y_train)
cat_preds = cat_model.predict(X_test.values)

cat_mae = mean_absolute_error(y_test, cat_preds)
cat_mse = mean_squared_error(y_test, cat_preds)
cat_rmse = np.sqrt(cat_mse)
cat_r2 = r2_score(y_test, cat_preds)

print("CatBoost Metrics:")
print(f"MAE: {cat_mae:.4f}, MSE: {cat_mse:.4f}, RMSE: {cat_rmse:.4f}, R2: {cat_r2:.4f}\n")

CatBoost Metrics:
MAE: 20.8844, MSE: 847.3816, RMSE: 29.1098, R2: 0.9917



In [35]:
ridge_model = Ridge(**ridge_params, random_state=42)
ridge_model.fit(X_train_scaled, y_train)
ridge_preds = ridge_model.predict(X_test_scaled)

ridge_mae = mean_absolute_error(y_test, ridge_preds)
ridge_mse = mean_squared_error(y_test, ridge_preds)
ridge_rmse = np.sqrt(ridge_mse)
ridge_r2 = r2_score(y_test, ridge_preds)

print("Ridge Regression Metrics:")
print(f"MAE: {ridge_mae:.4f}, MSE: {ridge_mse:.4f}, RMSE: {ridge_rmse:.4f}, R2: {ridge_r2:.4f}\n")

Ridge Regression Metrics:
MAE: 34.7915, MSE: 2156.2815, RMSE: 46.4358, R2: 0.9788



In [36]:
linear_model.fit(X_train_scaled, y_train)
linear_preds = linear_model.predict(X_test_scaled)

linear_mae = mean_absolute_error(y_test, linear_preds)
linear_mse = mean_squared_error(y_test, linear_preds)
linear_rmse = np.sqrt(linear_mse)
linear_r2 = r2_score(y_test, linear_preds)

print("Linear Regression Metrics:")
print(f"MAE: {linear_mae:.4f}, MSE: {linear_mse:.4f}, RMSE: {linear_rmse:.4f}, R2: {linear_r2:.4f}\n")

Linear Regression Metrics:
MAE: 34.7911, MSE: 2156.2272, RMSE: 46.4352, R2: 0.9788



In [37]:
knn_model = KNeighborsRegressor(**knn_params)
knn_model.fit(X_train_scaled, y_train)
knn_preds = knn_model.predict(X_test_scaled)

knn_mae = mean_absolute_error(y_test, knn_preds)
knn_mse = mean_squared_error(y_test, knn_preds)
knn_rmse = np.sqrt(knn_mse)
knn_r2 = r2_score(y_test, knn_preds)

print("KNN Metrics:")
print(f"MAE: {knn_mae:.4f}, MSE: {knn_mse:.4f}, RMSE: {knn_rmse:.4f}, R2: {knn_r2:.4f}\n")

KNN Metrics:
MAE: 7.0442, MSE: 336.4686, RMSE: 18.3431, R2: 0.9967

